In [ ]:
%load_ext autoreload
%autoreload 2

In [15]:
import sklearn.pipeline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import optuna
import pickle
import scipy
import yellowbrick

from statsmodels import api as sm

import sklearn
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, \
    QuadraticDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

import posture_detector
from posture_detector.constants import columns


In [ ]:
figure_path = '../images/'

In [4]:
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Load and parse dataset

In [5]:
data_location = 'data/raw/datalog_at_2021-07-04_19_07_28.038692'

In [6]:
df_raw = pd.read_csv('../' + data_location)
df_raw.head()

,Ax,Ay,Az,Temperature,gx,gy,gz,FlexSensor,Label
0,-0.187500,-0.883301,-0.452393,27.4947,1.90840,1.625950,-13.87790,562,NaN
1,-0.286377,-0.917725,-0.354248,27.4947,-4.64122,-1.694660,-4.77099,561,NaN
2,-0.350342,-0.951172,-0.354492,27.4947,-5.37405,-0.496183,1.30534,561,NaN
3,-0.303711,-0.922119,-0.364258,27.5888,-3.95420,0.015267,0.51145,561,NaN
4,-0.299561,-0.919678,-0.356934,27.4947,-2.96947,1.145040,5.68702,561,NaN


In [7]:
df = posture_detector.data.parse_dataset(df_raw)
df.head()

,Ax,Ay,Az,gx,gy,gz,FlexSensor,Label
982,-0.610840,-0.723877,-0.347656,-0.954198,2.297710,1.854960,573,0
983,-0.630127,-0.722168,-0.338867,-2.595420,1.358780,0.793893,572,0
984,-0.632568,-0.718506,-0.345459,-3.633590,0.656489,3.091600,572,0
985,-0.637451,-0.713135,-0.352051,-3.503820,1.648850,3.587790,574,0
986,-0.622559,-0.710938,-0.329834,-4.610690,-0.931298,2.435110,574,0


In [16]:
X, y = posture_detector.data.train_test.split_record_data(df.drop(columns=['gx', 'gy', 'gz']))
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = tts(X, y, train_size=2750)

# Logistic Regression

In [18]:
logit = sm.Logit(y_train, X_train).fit()


Optimization terminated successfully.
         Current function value: 0.077143
         Iterations 10


In [20]:
print(logit.summary())

                           Logit Regression Results                           
Dep. Variable:                  Label   No. Observations:                 2750
Model:                          Logit   Df Residuals:                     2746
Method:                           MLE   Df Model:                            3
Date:                Mon, 05 Jul 2021   Pseudo R-squ.:                  0.8887
Time:                        23:26:26   Log-Likelihood:                -212.14
converged:                       True   LL-Null:                       -1906.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.7403      0.285     -2.596      0.009      -1.299      -0.181
x2            -1.7357      0.237     -7.330      0.000      -2.200      -1.272
x3            -0.0155      0.155     -0.100      0.9

In [38]:
importances = list(map(
    lambda x: np.log(np.abs(x)),
    logit.params.values
))

# Odds Ratio

In [39]:
dict(zip(df.columns[[0, 1 ,2 , -2]], importances))


{'Ax': -0.30073504570849174,
 'Ay': 0.5513834372706636,
 'Az': -4.167273927892697,
 'FlexSensor': 2.2043484997935714}